# Visualization of DPS results in a map

In [ ]:
!pip install cogeo-mosaic

In [51]:
from glob import glob
import os
import json
import urllib
from typing import Dict

import folium
import geopandas as gpd
from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

In [9]:
def local_to_s3(url):
    return url.replace("/projects/my-private-bucket", "s3://maap-ops-workspace/nathanmthomas")
    
def s3_to_https(url):
    parts = url.split("/")
    return os.path.join('https://',f'{parts[2]}.s3.amazonaws.com', *parts[3:])

In [8]:
dps_files = '/projects/tile-index2.shp'
dps_output = gpd.read_file(dps_files)
dps_output.head()

,location,geometry
0,/projects/my-private-bucket/dps_output/do_topo...,"POLYGON ((4598522.000 7353304.000, 4508522.000..."
1,/projects/my-private-bucket/dps_output/do_topo...,"POLYGON ((4598522.000 7263304.000, 4508522.000..."
2,/projects/my-private-bucket/dps_output/do_topo...,"POLYGON ((4688522.000 7263304.000, 4598522.000..."
3,/projects/my-private-bucket/dps_output/do_topo...,"POLYGON ((4238522.000 7173304.000, 4148522.000..."
4,/projects/my-private-bucket/dps_output/do_topo...,"POLYGON ((4328522.000 7173304.000, 4238522.000..."


In [10]:
dps_output['s3'] = local_to_s3(dps_output['location'])

In [15]:
tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
tile_index = gpd.read_file(tile_index_path)

In [22]:
albers = tile_index.crs
dps_output.set_crs(albers, inplace=True)

In [30]:
dps_output_4236 = dps_output.to_crs("EPSG:4326")

In [37]:
m = folium.Map()
folium.GeoJson(dps_output_4236).add_to(m)
m

## Create MosaicJson

In [41]:
def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]



In [46]:
geojson = json.loads(dps_output_4236.to_json())

In [47]:
mosaicdata = MosaicJSON.from_features(geojson.get('features'), minzoom=8, maxzoom=16, accessor=get_accessor)

In [48]:
mosaic_json = "/projects/my-public-bucket/elev_check.json"
with MosaicBackend(mosaic_json, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)



## See it on a map

In [52]:
wmts_url = "https://h9su0upami.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = {
    "tile_format":"png",
    "tile_scale":"1",
    "pixel_selection":"first",
    "TileMatrixSetId":"WebMercatorQuad",
    "url":"https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/elev_check.json",
    "bidx":"3", # Select which band to use
    "resampling_method":"nearest",
    "rescale":"0,1", # Values in data are from 0 to 1
    "return_mask":"true",
    "colormap_name":"magma" # Any colormap from matplotlib will work
}

wmts_call = "?".join([wmts_url, urllib.parse.urlencode(params)])

# Note Jupyter bug add amp; incorrectly when printing the url
wmts_call

'https://h9su0upami.execute-api.us-east-1.amazonaws.com//mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=1&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=https%3A%2F%2Fmaap-ops-workspace.s3.amazonaws.com%2Fshared%2Fnathanmthomas%2Felev_check.json&bidx=3&resampling_method=nearest&rescale=0%2C1&return_mask=true&colormap_name=magma'

In [53]:
import ipycmc
w = ipycmc.MapCMC()
w

MapCMC()

In [54]:
# This adds a new layer to the map above, call Cloud Optimized GeoTIFF
w.load_layer_config(wmts_call, "wmts/xml")

